In [1]:
import numpy
print(numpy.__version__)

1.20.3


In [2]:
# numpy downgrade가 필요할 수 있음
!python -m pip install numpy==1.2

ERROR: Could not find a version that satisfies the requirement numpy==1.2
ERROR: No matching distribution found for numpy==1.2


In [3]:
!pip install ensemble_boxes

In [4]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [5]:
# ensemble csv files
submission_files = ['./ensemble_test/output_hwang.csv',
'./ensemble_test/output_Jun.csv',
'./ensemble_test/submission_ensemble_test.csv'] # submission lists
submission_df = [pd.read_csv(file) for file in submission_files]

In [6]:
image_ids = submission_df[0]['image_id'].tolist()

In [7]:
annotation = '../../dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [8]:
prediction_strings = []
file_names = []
iou_thr = 0.7

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [9]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_ensemble_test_2.csv')

submission.head()

,PredictionString,image_id
0,0 0.45086154 479.85797 592.0607 563.34784 665....,test/0000.jpg
1,0 0.7513934 765.1206 664.00977 898.19446 992.8...,test/0001.jpg
2,0 0.9994171857833862 880.2037 466.03952 1024.0...,test/0002.jpg
3,0 0.4029529392719269 154.2831573486328 259.012...,test/0003.jpg
4,0 0.9879557490348816 440.39398 406.09283 657.1...,test/0004.jpg
